In [6]:
import os
from pathlib import Path

import pandas as pd
import networkx as nx
import scipy.sparse as sp
import torch
import yaml

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

import mscproject.dataprep as dp
import mscproject.features as feat
import mscproject.simulate as sim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data_dir = "data/pyg/processed/components"

In [4]:
data_files = [f for f in Path(data_dir).iterdir() if f.is_file()]

In [7]:
df01 = torch.load(data_files[0])

In [12]:
df01.edge_stores

[{'edge_index': tensor([[9, 9, 8, 9, 8, 9, 9, 9],
         [4, 2, 0, 3, 5, 8, 7, 6]]), 'edge_attr': tensor([[1.0860],
         [1.0860],
         [1.0860],
         [1.0860],
         [1.0860],
         [1.0860],
         [1.0860],
         [1.0860]])},
 {'edge_index': tensor([[2, 0, 1, 0, 2, 1],
         [1, 1, 1, 9, 9, 9]]), 'edge_attr': tensor([[-0.9763],
         [-0.9763],
         [-0.9763],
         [-0.9763],
         [-0.9763],
         [-0.9763]])}]

In [16]:
data_dict = {}

for i, df in enumerate(data_files):
    df = torch.load(df)
    data_dict.update({i: (df.num_nodes, df.num_edges)})

In [17]:
df = pd.DataFrame.from_dict(data_dict, orient="index")

In [21]:
df.describe()

,0,1
count,6864.000000,6864.000000
mean,18.813957,19.642483
std,593.806444,645.307042
min,2.000000,1.000000
25%,9.000000,9.000000
50%,11.000000,10.000000
75%,13.000000,14.000000
max,49204.000000,53470.000000


In [23]:
df.sort_values(by=0, ascending=False)

,0,1
5800,49204,53470
6068,107,153
5325,102,112
2645,99,106
843,98,107
...,...,...
6026,2,1
4958,2,1
3178,2,1
6033,2,1
